## Inclusion of covariates

This notebook generates a covariate field then uses the field to generate wildebeest abundance maps.



In [ ]:
import numpy as np
import gstools as gs
import matplotlib.pyplot as plt
from math import *
from tqdm import tqdm
import pandas as pd
from scipy import interpolate

In [2]:
vals = np.arange(2,3) # look at effect of covariate for p=0.2
Nr = 10 # n repeats

In [3]:
for r in tqdm(range(Nr)):

    for v in vals:
        p = 0.1*v
        
        # make a random field with 2k correlation length
        x = y = np.arange(0,50000,100)
        L = 2000
        var=0.1

        model = gs.Exponential(dim=2, var=var, len_scale=L)
        srf = gs.SRF(model)
        field = srf.structured([x, y])

        
        # make a random field with 2k correlation length
        x = y = np.arange(0,50000,100)
        L = 2000
        var=0.9

        model = gs.Gaussian(dim=2, var=var, len_scale=L)
        srf = gs.SRF(model)
        cvfield = srf.structured([x, y])

        # make a 5k presence-absence field
        L = 4000
        var = 1

        model = gs.Gaussian(dim=2, var=var, len_scale=L)
        srf = gs.SRF(model)
        
        # we make the pa field bigger so we can centre it on the largest cluster of wildebeest
        x2 = y2 = np.arange(-50000,100000,1000)
        pa_field = srf.structured([x2, y2])
        # interpolate as the grid generation is slow for large N
        f = interpolate.interp2d(x2, y2, pa_field, kind='cubic')
        x2 = y2 = np.arange(-50000,100000,100)

        pa_field = f(x2,y2)

        # next we find the highest value of the pa field
        ij = pa_field[500:1000,500:1000].argmax()
        # and centre it so it's at the middle of the survey
        i = ij//500
        j = ij - i*500
        pa_field2 = pa_field[500+i-250:500+i+250,500+j-250:500+j+250]

        # only populate the top p%
        cutoff = np.percentile(pa_field2,100*(1-p))

        # set to negative infinity sets it to zero when we take the exponential
        field[pa_field2<cutoff]=-np.inf
        
        #set the mean so we end up with roughly 1.3m wildebeest
        target_w = 1300000
        vf = np.var(field[np.isfinite(field)]+cvfield[np.isfinite(field)])
        mf = np.mean(field[np.isfinite(field)]+cvfield[np.isfinite(field)])
        target_log_density = np.log(target_w/(p*50000*50000))-0.5*vf-mf
        
        # generate a random count for each grid cell of 100mx100m
        wcounts = np.random.poisson(lam = 100*100*np.exp(target_log_density+field+cvfield))

        # save the covariate field
        outputfile = '../data/simdata/cv_field_p' + str(v) + '_r' + str(r) + '.npy'
        np.save(outputfile,cvfield)

        # next place the wildebeest in a random location within their cell
        total = wcounts.sum()
        positions = np.zeros((total,2))

        min_x = 0
        max_x = 50000
        min_y = 0
        max_y = 50000
        #print(total)
        w = 0
        for i in (range(len(x))):
            for j in range(len(y)):
                wtotal =  wcounts[i,j]
                positions[w:w+wtotal,0]=x[i]+np.random.uniform(100,size=wtotal) # place at random in the 100x100 cell
                positions[w:w+wtotal,1]=y[j]+np.random.uniform(100,size=wtotal)
                w = w + wtotal

        # fly the survey
        yvals = np.arange(positions[:,1].min(),positions[:,1].max(),2500)
        yvals = np.arange(135,max_y,2500)
        photos=[]
        transect=0
        for yv in (yvals):

            ymin = yv - 135
            ymax = yv + 135
            xvals = np.arange(90,max_x,500)

            if len(xvals):
                transect+=1

            for xv in xvals:

                xmin = xv - 90
                xmax = xv + 90
                
                ymin = ymin 
                ymax = ymax

                count = np.sum((positions[:,0]>xmin)&(positions[:,0]<xmax)&(positions[:,1]>ymin)&(positions[:,1]<ymax))

                photo = [xv,yv,count,transect]
                photos.append(photo)


        # save the data
        outputfile = 'simdata/cv_simdata_p' + str(v) + '_r' + str(r) + '.csv'
        f = open(outputfile,'w')
        f.write("x,y,wildebeest,transect_id,photo_area\n")

        x_str = '%.2f,' % 0
        y_str = '%.2f,' % 0
        w_str = '%d,' % total
        t_str = '%d,' % -1
        a_str = '%d' % 0
        f.write(x_str + y_str + w_str + t_str + a_str )
        f.write('\n')
        for photo in photos:
            x_str = '%.2f,' % (photo[0]/1)
            y_str = '%.2f,' % (photo[1]/1)
            w_str = '%d,' % photo[2]
            t_str = '%d,' % photo[3]
            a_str = '%.6f' % (180*270/1/1)
            f.write(x_str + y_str + w_str + t_str + a_str )
            f.write('\n')
        f.close()


100%|██████████| 10/10 [18:09<00:00, 108.93s/it]
